In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [3]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install

**Весь код выполняется за 40 сек на intel core i5 двухлетней давности \
Внизу заготовка на импорт модели**

In [5]:
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, RobustScaler, OneHotEncoder, PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import make_scorer
from sklearn.svm import SVR
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_squared_error
import pickle

### Область работы 2 (выполнение лучшей модели)

In [7]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения


In [8]:
# Путь к тренировочному набору
path_train = 'train.csv' # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  = 'test.csv' # содержит только имя файла, без имен папок

In [9]:
df = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)
 
df = df.drop_duplicates()

df.loc[df['x'] <= 0, 'x'] = np.nan
df.loc[df['y'] <= 0, 'y'] = np.nan
df.loc[df['z'] <= 0, 'z'] = np.nan
df.loc[df['carat'] <= 0, 'carat'] = np.nan
df.loc[df['depth'] <= 0, 'depth'] = np.nan
df.loc[df['table'] <= 0, 'table'] = np.nan
df.loc[df['x'] > 20, 'x'] /= 10
df.loc[df['y'] > 20, 'y'] /= 10
df.loc[df['z'] > 20, 'z'] /= 10
df['depth'] = round(2 * df['z'] / (df['x'] + df['y']) * 100, 1)


df_test.loc[df_test['x'] <= 0, 'x'] = np.nan
df_test.loc[df_test['y'] <= 0, 'y'] = np.nan
df_test.loc[df_test['z'] <= 0, 'z'] = np.nan
df_test.loc[df_test['carat'] <= 0, 'carat'] = np.nan
df_test.loc[df_test['depth'] <= 0, 'depth'] = np.nan
df_test.loc[df_test['table'] <= 0, 'table'] = np.nan
df_test.loc[df_test['x'] > 20, 'x'] /= 10
df_test.loc[df_test['y'] > 20, 'y'] /= 10
df_test.loc[df_test['z'] > 20, 'z'] /= 10
df_test['depth'] = round(2 * df_test['z'] / (df_test['x'] + df_test['y']) * 100, 1)

In [10]:
def irq_mse(y_true, y_pred):
    delta = y_pred - y_true
    quant_75 = np.quantile(delta, 0.75)
    quant_25 = np.quantile(delta, 0.25)
    irq = quant_75 - quant_25
    mask = (delta < quant_25 - 1.5*irq) | (delta > quant_75 + 1.5*irq)
    
    if mask.sum() == 0:
        irq_mse = 0
    else:
        delta_new = delta[mask]**2
        irq_mse = -1/len(delta_new) * np.sum(delta_new)
    return irq_mse

score_irq_mse = make_scorer(irq_mse)
score_irq_mse

make_scorer(irq_mse, response_method='predict')

In [11]:
X = df.drop('price', axis=1)
y = df['price']

In [12]:
cat_features = ['cut', 'color', 'clarity']
axis_features = ['x', 'y', 'z', 'carat']

axis_transformer = Pipeline(steps=[
    ('missing_num', IterativeImputer(missing_values=np.nan, max_iter=20)),
    ('polynom', PolynomialFeatures(2, include_bias=False)),
    ('scaler', RobustScaler())
])

cat = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

CT = ColumnTransformer([
    ("axis_transformer", axis_transformer, axis_features),
    ("cat", cat, cat_features)
]).set_output(transform='pandas')

pipeline = Pipeline(steps=[
    ('preproc', CT),
    ('estimator', SVR())
])

model_target = TransformedTargetRegressor(
    regressor=pipeline,
    transformer=PowerTransformer(method='yeo-johnson')
)

In [13]:
model_target.fit(X, y)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\impute\_iterative.py:825: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


TransformedTargetRegressor(regressor=Pipeline(steps=[('preproc',
                                                      ColumnTransformer(transformers=[('axis_transformer',
                                                                                       Pipeline(steps=[('missing_num',
                                                                                                        IterativeImputer(max_iter=20)),
                                                                                                       ('polynom',
                                                                                                        PolynomialFeatures(include_bias=False)),
                                                                                                       ('scaler',
                                                                                                        RobustScaler())]),
                                                                                       ['x',
                                                                                        'y',
                                                                                        'z',
                                                                                        'carat']),
                                                                                      ('cat',
                                                                                       Pipeline(steps=[('imputer',
                                                                                                        SimpleImputer(strategy='most_frequent')),
                                                                                                       ('onehot',
                                                                                                        OneHotEncoder(handle_unknown='ignore',
                                                                                                                      sparse_output=False))]),
                                                                                       ['cut',
                                                                                        'color',
                                                                                        'clarity'])])),
                                                     ('estimator', SVR())]),
                           transformer=PowerTransformer())

In [14]:
y_predict = model_target.predict(df_test)

In [15]:
print(len(y_predict))
print(df_test.shape)

13485
(13485, 9)


In [16]:
# Вектора предсказанных значений  y_predict полученый на основане тестового набора
y_predict

array([8641.4539582 , 1523.53186308, 2786.1476853 , ..., 2475.52787717,
       1681.04350208, 1881.12942691])

In [17]:
# # Запись модели
# with open('model_target', 'wb') as f:
#     pickle.dump(model_target, f)

In [18]:
# # Импорт модели
# with open('model_target', 'rb') as f:
#     model = pickle.load(f)

In [19]:
# y_predict = model.predict(df_test)

In [20]:
# y_predict